In [1]:
import numpy as np

In [2]:
import os

In [3]:
from updown_data import updown_z

In [4]:
from export_train_test import export_train_validation_test

for i in range(100):
    updown_z(i, "aafiles/Jpp_v6_aa_files/Xy_files/Xy_numu_"+str(i+1)+"_sel5_doms.npz", 
             "aafiles/Jpp_v6_aa_files/Xy_files/numu_"+str(i+1)+"_sel5_dir_z.npz",
            "Xy_files/updown/", "Xy_numu")

for i in range(100):
    updown_z(i, "aafiles/Jpp_v6_aa_files/Xy_files/Xy_nue_"+str(i+1)+"_sel5_doms.npz", 
             "aafiles/Jpp_v6_aa_files/Xy_files/nue_"+str(i+1)+"_sel5_dir_z.npz",
            "Xy_files/updown/", "Xy_nue")

folder = os.path.join("Xy_files", "updown")
for i in range(100):
    export_train_validation_test(i, os.path.join(folder,"Xy_numu{}_sel5_doms_updown_z.npz".format(i+1)),
                            os.path.join(folder,"Xy_nue{}_sel5_doms_updown_z.npz".format(i+1)),
                            out_dir = os.path.join("train_test_files","updown_z"))
    print("file {} completed".format(i+1))

In [4]:
import matplotlib.pyplot as plt

% matplotlib inline

In [5]:
from keras.layers import Input
from keras.layers import concatenate, add
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.core import Flatten, Dense

from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta


Using TensorFlow backend.


In [6]:
from get_iterations import get_n_iterations, get_class_weights

In [7]:
from multi_files_generator import Xy_gen_TZ as generator_tz

In [18]:
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

In [8]:
train_test_dir = os.path.join("train_test_files", "updown_z")

fnames_train = np.asarray([os.path.join(train_test_dir, "Xy_train{}_sel5_doms.npz".format(i+1)) for i in range(100)])
fnames_test = np.asarray([os.path.join(train_test_dir, "Xy_test{}_sel5_doms.npz".format(i+1)) for i in range(100)])
fnames_validation = np.asarray([os.path.join(train_test_dir, "Xy_val{}_sel5_doms.npz".format(i+1)) for i in range(100)])

#fnames_indx = np.asarray(["train_test_files/Xy_indx"+str(i+1)+"_sel5_updown.npz"for i in range(100)])

In [15]:
n_files = 100
steps_per_epoch, n_events = get_n_iterations(fnames_train[:n_files])
print(steps_per_epoch, n_events)
validation_steps, n_evts_val = get_n_iterations(fnames_validation[:n_files])
print(validation_steps, n_evts_val)

prediction_steps, n_evts_test = get_n_iterations(fnames_test[:n_files])
print(prediction_steps, n_evts_test)

get_class_weights(fnames_train[:n_files])
cls_weights = {i:v for i, v in enumerate(get_class_weights(fnames_train[:n_files]))}


(2356, 150781)
(590, 37749)
(738, 47181)


In [16]:
def vgg_tz(num_classes, optimizer, include_dense=False):
    """VGG inspired Convolutional Networks with multiple inputs
    
    Parameters
    ----------
    num_classes : int
        Number of classes to predict
            
    optimizer : keras.optimizers.Optimizer
        Instance of Keras optimizer to attach to the resulting network
        
    include_dense : bool (default False)
        Flag indicating wether to include a Dense Layer at the top 
        of each separate vgg-like sub-network.
    """
    
    def _conv_block(input_layer, include_dense=False, pooling_size=(2, 2)):
        """VGG-inspired Convolutional Blocks"""
        
        prefix = input_layer.name.split('_')[0]

        # Block 1
        x = Conv2D(32, kernel_size=(12, 12), activation='relu', 
                   padding='same', name='{}_block1_conv1'.format(prefix))(input_layer)
        
        x = Conv2D(32, kernel_size=(12, 12), activation='relu', 
                   padding='same', name='{}_block1_conv2'.format(prefix))(x)
        
        x = AveragePooling2D(pool_size=pooling_size, strides=(2, 2), 
                         padding='same', name='{}_block1_pool'.format(prefix))(x)

        # Block 2
        x = Conv2D(64, kernel_size=(12, 12), activation='relu', 
                   padding='same', name='{}_block2_conv1'.format(prefix))(x)
        
        x = Conv2D(64, kernel_size=(12, 12), activation='relu', 
                   padding='same', name='{}_block2_conv2'.format(prefix))(x)
        
        x = AveragePooling2D(pool_size=pooling_size, strides=(2, 2), 
                         padding='same', name='{}_block2_pool'.format(prefix))(x)

        # Block 3
        x = Conv2D(128, kernel_size=(12, 12), activation='relu', 
                   padding='same', name='{}_block3_conv2'.format(prefix))(x)
        
        x = AveragePooling2D(pool_size=pooling_size, strides=(2, 2), 
                             padding='same', name='{}_block3_pool'.format(prefix))(x)
        x = Flatten()(x)
        if include_dense:
            x = Dense(512)(x)
        return x


    ZT_SHAPE = (1, 75, 18)
    zt_layer = Input(shape=ZT_SHAPE, name='zt_input')
    
    zt_branch = _conv_block(zt_layer, pooling_size=(6, 6))#, include_dense)
        
    x = Dense(512, activation='relu', name='fc-1')(zt_branch)
    x = Dense(512, activation='relu', name='fc-2')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=zt_layer, outputs=predictions, name='vgglike_tz')
    model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])
    return model

In [20]:
b_size = 64
epochs = 100
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_tz_net = vgg_tz(**fargs)

fitargs = {'steps_per_epoch': steps_per_epoch, 'epochs': epochs, 'verbose': 1, 'shuffle': True,
           'class_weight' : {i:v for i, v in enumerate(get_class_weights(fnames_train[:n_files]))},
           "callbacks": [#TensorBoard(batch_size=b_size, histogram_freq=5, embeddings_freq=5,
                         #           embeddings_layer_names=["zt_block1_conv1", "zt_block1_conv2"]), 
                         ModelCheckpoint("models/vgg_tz_updown_z_cls_weighted_"+str(n_files)+"files_"+str(b_size)+"_12x12_"+str(epochs)+"epochs.hdf5",
                                                        save_best_only=True),
                         EarlyStopping(monitor="val_loss", patience=5)]}
history_vgg_tz = vgg_tz_net.fit_generator(generator_tz(fnames_train[:n_files], batch_size=b_size), 
                                          validation_data=generator_tz(fnames_validation[:n_files]),
                                          validation_steps=validation_steps, **fitargs)




Epoch 1/100
2356/2356 [==============================] - 404s - loss: 0.0876 - acc: 0.9600 - val_loss: 0.0119 - val_acc: 0.9966
Epoch 2/100
2356/2356 [==============================] - 404s - loss: 0.0080 - acc: 0.9976 - val_loss: 0.0044 - val_acc: 0.9988
Epoch 3/100
2356/2356 [==============================] - 407s - loss: 0.0043 - acc: 0.9988 - val_loss: 0.0028 - val_acc: 0.9993
Epoch 4/100
2356/2356 [==============================] - 401s - loss: 0.0026 - acc: 0.9992 - val_loss: 0.0016 - val_acc: 0.9995
Epoch 5/100
2356/2356 [==============================] - 394s - loss: 0.0019 - acc: 0.9994 - val_loss: 0.0035 - val_acc: 0.9993
Epoch 6/100
2356/2356 [==============================] - 389s - loss: 0.0016 - acc: 0.9996 - val_loss: 0.0023 - val_acc: 0.9994
Epoch 7/100
2356/2356 [==============================] - 388s - loss: 0.0013 - acc: 0.9996 - val_loss: 0.0039 - val_acc: 0.9991
Epoch 8/100
2356/2356 [==============================] - 400s - loss: 0.0011 - acc: 0.9997 - val_loss: 0

In [21]:
y_true = list()
y_pred = list()

predict_steps, n_test_events = get_n_iterations(fnames_test[:n_files], 64)
print(predict_steps, n_test_events)

data_gen = generator_tz(fnames_test[:n_files], batch_size=64)
for i in range(predict_steps):
    ZT_batch, Y_batch_true = next(data_gen)
    Y_batch_pred = vgg_tz_net.predict_on_batch(ZT_batch)
    y_batch_pred = np.argmax(Y_batch_pred, axis=1)
    y_batch_true = np.argmax(Y_batch_true, axis=1)
    y_true.append(y_batch_true)
    y_pred.append(y_batch_pred)
    
y_true = np.hstack(np.asarray(y_true))
y_pred = np.hstack(np.asarray(y_pred))

(738, 47181)


In [22]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_true, y_pred)
print(score)


0.999597295522


## Testing Logistic Regression

## Model MLP / Logistic Regression-like

In [27]:
from keras.layers.core import Reshape

In [29]:
ZT_SHAPE = (1, 75, 18)
zt_layer = Input(shape=ZT_SHAPE, name='zt_input')
reshape = Reshape(target_shape=(75*18,))(zt_layer)
x = Dense(512, activation='sigmoid', name='logistic')(reshape)
predictions = Dense(2, activation='softmax')(x)

mlp_logistic = Model(inputs=zt_layer, outputs=predictions, name='mlp_logistic')
mlp_logistic.compile(loss=categorical_crossentropy, optimizer=Adadelta(), metrics=['accuracy'])
    

In [30]:
mlp_logistic.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zt_input (InputLayer)        (None, 1, 75, 18)         0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 1350)              0         
_________________________________________________________________
logistic (Dense)             (None, 512)               691712    
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 1026      
Total params: 692,738
Trainable params: 692,738
Non-trainable params: 0
_________________________________________________________________


In [ ]:
fitargs = {'steps_per_epoch': steps_per_epoch, 'epochs': epochs, 'verbose': 1, 'shuffle': True,
           "callbacks": [#TensorBoard(batch_size=b_size, histogram_freq=5, embeddings_freq=5,
                         #           embeddings_layer_names=["zt_block1_conv1", "zt_block1_conv2"]), 
                         ModelCheckpoint("models/mlp_logistic"+str(n_files)+"files_"+str(b_size)+"_12x12_"+str(epochs)+"epochs.hdf5",
                                                        save_best_only=True),
                         EarlyStopping(monitor="val_loss", patience=5)]}
history_mlp_logistic = mlp_logistic.fit_generator(generator_tz(fnames_train[:n_files], batch_size=b_size), 
                                          validation_data=generator_tz(fnames_validation[:n_files]),
                                          validation_steps=validation_steps, **fitargs)


Epoch 1/100
2356/2356 [==============================] - 343s - loss: 0.5479 - acc: 0.7300 - val_loss: 0.5127 - val_acc: 0.7621
Epoch 2/100
2356/2356 [==============================] - 339s - loss: 0.4896 - acc: 0.7759 - val_loss: 0.4737 - val_acc: 0.7874
Epoch 3/100
2356/2356 [==============================] - 341s - loss: 0.4536 - acc: 0.7977 - val_loss: 0.4445 - val_acc: 0.8052
Epoch 4/100
2356/2356 [==============================] - 343s - loss: 0.4256 - acc: 0.8123 - val_loss: 0.4201 - val_acc: 0.8181
Epoch 5/100
2356/2356 [==============================] - 344s - loss: 0.4017 - acc: 0.8240 - val_loss: 0.3981 - val_acc: 0.8285
Epoch 6/100
2356/2356 [==============================] - 343s - loss: 0.3800 - acc: 0.8347 - val_loss: 0.3777 - val_acc: 0.8399
Epoch 7/100
2356/2356 [==============================] - 343s - loss: 0.3597 - acc: 0.8446 - val_loss: 0.3584 - val_acc: 0.8503
Epoch 8/100
2356/2356 [==============================] - 344s - loss: 0.3406 - acc: 0.8548 - val_loss: 0

In [32]:
n_files

100

In [33]:
mlp_logistic

In [34]:
y_true = list()
y_pred = list()

predict_steps, n_test_events = get_n_iterations(fnames_test[:n_files], 64)
print(predict_steps, n_test_events)

data_gen = generator_tz(fnames_test[:n_files], batch_size=64)
for i in range(predict_steps):
    ZT_batch, Y_batch_true = next(data_gen)
    Y_batch_pred = mlp_logistic.predict_on_batch(ZT_batch)
    y_batch_pred = np.argmax(Y_batch_pred, axis=1)
    y_batch_true = np.argmax(Y_batch_true, axis=1)
    y_true.append(y_batch_true)
    y_pred.append(y_batch_pred)
    
y_true = np.hstack(np.asarray(y_true))
y_pred = np.hstack(np.asarray(y_pred))

(738, 47181)


In [35]:
score = accuracy_score(y_true, y_pred)
print(score)

0.91439350586
